<a href="https://colab.research.google.com/github/arpitvaghela/teamup-recommender/blob/main/CollabFilter_Grp_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from typing import Set,List,Tuple

# Collaborative Filtering 


In [ ]:
# data for Basic CF
df = pd.DataFrame({
    "r1":[1,0,0],
    "r2":[1,1,0],
    "r3":[1,1,1],
    "r4":[0,1,1],
},
    index = ["s1","s2","u"]
)
df

,r1,r2,r3,r4
s1,1,1,1,0
s2,0,1,1,1
u,0,0,1,1


In [ ]:
def get_recipient(u:str)-> Set[str]:
    """Get recipient of user u
    
    Args:
        u : a sender user
    
    Returns:
        Set of recipient user of user u
    """
    try:
        u_row = df.loc[u]
        return set(u_row.loc[u_row == 1].index)
    except KeyError:
        return set()

get_recipient("u")

{'r3', 'r4'}

In [ ]:

def similar_recipient(r:str)-> Set[str]:
    """Get similar recipients of user r

    Args: 
        r : a recipient user 
    
    Returns: 
        Set of similar recipient

    Note:
        r ~r r' => w -> r & w -> r'
    """
    try:
        df_r = df.loc[df[r] == 1 ] # df where r is a recipient
    except KeyError:
        return set()
    
    list_of_similar_r = df_r.apply(lambda r: r.loc[r == 1 ].index,axis=1) # list of similar recipient in each row 
    
    S = set() # create a set of recipients 
    
    for l in list_of_similar_r:
        S |= set(l)
    
    try:
        S.remove(r)
    except KeyError:
        pass
    
    return S


similar_recipient("u")

set()

In [ ]:
def similar_sender(s:str) -> Set[str]:
    """Get similar sender of user s
    
    Args:
        s : a sender user
    
    Returns :
        Set of similar sender
    
    Note :
        s ~s s' =>  s -> w & s' -> w
    """
    R = get_recipient(s)    # recipients of s
    df_s = df.loc[:,R]      # dataframe where s is a sender
    df_s = df_s[(df_s == 1).any(axis=1)]
    
    S = set(df_s.index)     # set of similar senders
    try:
        S.remove(s)             # remove current user
    except KeyError:
        pass
    
    return S

similar_sender("u")

{'s1', 's2'}

In [ ]:
def Basic_CF(u:str) -> List[Tuple[str,int]]:
    """Recommend similar recipient of the recipient of u
    
    Args:
        u : user
    
    Returns:
        Sorted list of (user,vote) Tuples of recommendation
    
    Note:
        C = {<u,r> : (u -> r' & r' ~r r & u -/> r & r -/> u )}
        votes(u,r) = |{r' : u -> r' & r' ~r r }|
    """ 

    votes = {r:0 for r in df.columns} # initial votes are 0
    
    R1 = get_recipient(u) # set of recipients
    for r1 in R1:
        R = similar_recipient(r1) # set of similar recipients
        for r in R:
            if r not in R1 and u not in get_recipient(r):   # increment votes[r] if u -/> r and r -/> u 
                votes[r] += 1
                
    return sorted(list(votes.items()),key=lambda e:e[1],reverse=True) # return Soreted Tuple 

Basic_CF("u")

[('r2', 2), ('r1', 1), ('r3', 0), ('r4', 0)]

In [ ]:

# inverted CF recipient
df = pd.DataFrame({
    "s1":[0,0,0,0,0,0,0],
    "s2":[0,0,0,0,0,0,0],
    "r1":[1,0,0,0,0,0,0],
    "r2":[1,1,0,0,0,0,0],
    "r3":[0,0,1,0,0,0,0],
    "r4":[0,0,0,1,0,0,0],
    "u" :[1,1,0,0,0,0,0],
},
index = ["s1","s2","r1","r2","r3","r4","u"]
)
df

,s1,s2,r1,r2,r3,r4,u
s1,0,0,1,1,0,0,1
s2,0,0,0,1,0,0,1
r1,0,0,0,0,1,0,0
r2,0,0,0,0,0,1,0
r3,0,0,0,0,0,0,0
r4,0,0,0,0,0,0,0
u,0,0,0,0,0,0,0


In [ ]:
def Inverted_CF_Recipient(u:str) -> List[Tuple[str,int]]:
    """Recommend recipient of similar recipient of u
    Args:
        u : user
    Returns:
        Sorted list of (user.votes) Tuple of recommendation
    Note:
        C = {<u,r> : (r' ~r u & r' -> r & u -/> r & r -/> u )}
        votes(u,r) = |r' : { r' ~r u & r' -> r }|
    """
    votes = {r:0 for r in df.columns} # initial votes are 0

    U1 = get_recipient(u)

    R1 = similar_recipient(u) 
    for r1 in R1:
        R = get_recipient(r1)
        for r in R:
            if r not in U1 and u not in get_recipient(r):
                votes[r] += 1

    return sorted(list(votes.items()),key=lambda e:e[1],reverse=True) # return Soreted Tuple

Inverted_CF_Recipient("u")

[('r3', 1), ('r4', 1), ('s1', 0), ('s2', 0), ('r1', 0), ('r2', 0), ('u', 0)]

In [ ]:
# inverted CF sender
df = pd.DataFrame({
    "u" :[0,0,0,0,0,0,0],
    "s1":[1,0,0,0,0,0,0],
    "s2":[0,0,0,0,0,0,0],
    "s3":[0,0,0,0,0,0,0],
    "r1":[0,1,0,0,0,0,0],
    "r2":[0,1,1,0,0,0,0],
    "r3":[0,1,1,1,0,0,0]
},index=["u","s1","s2","s3","r1","r2","r3"])
df

,u,s1,s2,s3,r1,r2,r3
u,0,1,0,0,0,0,0
s1,0,0,0,0,1,1,1
s2,0,0,0,0,0,1,1
s3,0,0,0,0,0,0,1
r1,0,0,0,0,0,0,0
r2,0,0,0,0,0,0,0
r3,0,0,0,0,0,0,0


In [ ]:
def Inverted_CF_Sender(u:str)-> List[Tuple[str,int]]:
    """Recommend similar sender of recipient of u
     Args:
        u : User
    Returns:
        Sorted list of (user.votes) Tuple of recommendation
    Note:
        C = {<u,s> : (u -> s' & s' ~s s & u -/> s & s -/> u )}
        votes(u,s) = |s' : { s' ~s s & u -> s' }|
     """

    votes = {r:0 for r in df.columns} # initial votes are 0
    S1 = get_recipient(u)
    for s1 in S1:
        S = similar_sender(s1)
        for s in S:
            if s not in S1 and u not in get_recipient(s):
                 votes[s] += 1
    return sorted(list(votes.items()),key=lambda e:e[1],reverse=True) # return Soreted Tuple

Inverted_CF_Sender("u")

[('s2', 1), ('s3', 1), ('u', 0), ('s1', 0), ('r1', 0), ('r2', 0), ('r3', 0)]

In [ ]:
def Best_Two_CF(u:str) -> List[Tuple[str,int]]:
    """Best Two CF = Basic CF + Inverted CF Recipient
    Args:
        u : User
    Returns:
        Sorted list of (user.votes) Tuple of recommendation
    
    """
    votes = {r:0 for r in df.columns} # initial votes are 0

    # basic CF
    R1 = get_recipient(u) # set of recipients
    for r1 in R1:
        R = similar_recipient(r1) # set of similar recipients
        for r in R:
            if r not in R1 and u not in get_recipient(r):   # increment votes[r] if u -/> r and r -/> u 
                votes[r] += 1
                
    # inverted cf recipient
    S1 = get_recipient(u)
    for s1 in S1:
        S = similar_sender(s1)
        for s in S:
            if s not in S1 and u not in get_recipient(s):
                 votes[s] += 1
    
    return sorted(list(votes.items()),key=lambda e:e[1],reverse=True) # return Soreted Tuple

Best_Two_CF("u")

[('s2', 1), ('s3', 1), ('u', 0), ('s1', 0), ('r1', 0), ('r2', 0), ('r3', 0)]